<a href="https://colab.research.google.com/github/andres4ramos/167/blob/main/12_1_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.1: Self Attention**

This notebook builds a self-attention mechanism from scratch, as discussed in section 12.2 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [1]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [3]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 3
# Number of dimensions of each input
D = 4
# Create an empty list
all_x = []
# Create elements x_n and append to list
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
# Print out the list
print(all_x)


[array([[ 1.78862847],
       [ 0.43650985],
       [ 0.09649747],
       [-1.8634927 ]]), array([[-0.2773882 ],
       [-0.35475898],
       [-0.08274148],
       [-0.62700068]]), array([[-0.04381817],
       [-0.47721803],
       [-1.31386475],
       [ 0.88462238]])]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [4]:
# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

Now let's compute the queries, keys, and values for each input

In [34]:
all_queries = []
all_keys = []
all_values = []

for x in all_x:
    query = np.dot(omega_q, x) + beta_q.flatten()
    key = np.dot(omega_k, x) + beta_k.flatten()
    value = np.dot(omega_v, x) + beta_v.flatten()

    all_queries.append(query)
    all_keys.append(key)
    all_values.append(value)

We'll need a softmax function (equation 12.5) -- here, it will take a list of arbitrary numbers and return a list where the elements are non-negative and sum to one


In [35]:
def softmax(items_in):
    # Shift items_in by its maximum to prevent overflow
    exp_items = np.exp(items_in - np.max(items_in))
    # Compute softmax output
    items_out = exp_items / np.sum(exp_items)

    return items_out

Now compute the self attention values:

In [37]:
all_x_prime = []

for n in range(N):
    all_km_qn = []
    for key in all_keys:
        dot_product = np.dot(all_queries[n], key)
        all_km_qn.append(dot_product)

    attention = softmax(all_km_qn)
    print("Attentions for output ", n)
    print(attention)

    x_prime = np.zeros_like(all_values[0])
    for i, value in enumerate(all_values):
        x_prime += attention[i] * value

    all_x_prime.append(x_prime)

print("x_prime_0_calculated:", all_x_prime[0].transpose())
print("x_prime_0_true: [[ 0.94744244 -0.24348429 -0.91310441 -0.44522983]]")
print("x_prime_1_calculated:", all_x_prime[1].transpose())
print("x_prime_1_true: [[ 1.64201168 -0.08470004  4.02764044  2.18690791]]")
print("x_prime_2_calculated:", all_x_prime[2].transpose())
print("x_prime_2_true: [[ 1.61949281 -0.06641533  3.96863308  2.15858316]]")


Attentions for output  0
[[[2.88201344e-17 1.03158450e-15 2.18850584e-18 1.91057659e-19]
  [4.62896540e-07 3.30692046e-10 8.55616425e-05 1.19246540e-02]
  [3.40812297e-22 2.26642230e-18 5.99926383e-25 1.48829296e-27]
  [2.94734043e-13 1.50215843e-13 4.79001215e-13 7.58286586e-13]]

 [[2.40927759e-11 8.62373985e-10 1.82952584e-12 1.59718526e-13]
  [3.31110125e-15 2.36544184e-18 6.12022853e-13 8.52971093e-11]
  [1.76341036e-09 1.17267851e-05 3.10410277e-12 7.70063531e-15]
  [7.32126296e-13 3.73139688e-13 1.18985029e-12 1.88360172e-12]]

 [[3.68249604e-09 1.31810830e-07 2.79636589e-10 2.44124148e-11]
  [1.06222536e-18 7.58850948e-22 1.96341383e-16 2.73639331e-14]
  [1.48544316e-04 9.87828649e-01 2.61480161e-07 6.48678060e-10]
  [6.57684302e-13 3.35199154e-13 1.06886730e-12 1.69207866e-12]]]
Attentions for output  1
[[[1.18365030e-18 1.64535585e-15 6.43586474e-21 4.63981909e-23]
  [1.07504087e-13 7.78321858e-13 2.58208404e-14 6.69926054e-15]
  [6.21411778e-16 4.82787939e-14 2.69977494e-17 

Now let's compute the same thing, but using matrix calculations.  We'll store the $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ in the columns of a $D\times N$ matrix, using equations 12.6 and 12.7/8.

Note:  The book uses column vectors (for compatibility with the rest of the text), but in the wider literature it is more normal to store the inputs in the rows of a matrix;  in this case, the computation is the same, but all the matrices are transposed and the operations proceed in the reverse order.

In [21]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [28]:
def self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
    """
    Compute self-attention mechanism in matrix form.

    Parameters:
    - X : Input matrix (NxD) where N is the number of inputs, D is the dimension of each input.
    - omega_v, omega_q, omega_k : Weight matrices for values, queries, and keys, respectively.
    - beta_v, beta_q, beta_k : Bias vectors for values, queries, and keys, respectively.

    Returns:
    - X_prime : Output matrix after applying self-attention (NxD).
    """

    # Step 1: Compute queries, keys, and values
    queries = X @ omega_q + beta_q   # Shape: (N, D)
    keys = X @ omega_k + beta_k      # Shape: (N, D)
    values = X @ omega_v + beta_v    # Shape: (N, D)

    # Step 2: Compute dot products between queries and keys (Attention Scores)
    # Transpose keys to get the correct matrix multiplication shape
    attention_scores = queries @ keys.T   # Shape: (N, N)

    # Step 3: Apply softmax to calculate attention weights
    attention_weights = softmax(attention_scores)  # Shape: (N, N)

    # Step 4: Compute weighted sum of values based on attention weights
    X_prime = attention_weights @ values  # Shape: (N, D)

    return X_prime

In [40]:
# Copy data into matrix
X = np.zeros((D, N))
X[:, 0] = np.squeeze(all_x[0])
X[:, 1] = np.squeeze(all_x[1])
X[:, 2] = np.squeeze(all_x[2])

# Transpose X to match (N, D) shape
X = X.T

# Run the self-attention mechanism
X_prime = self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)


ValueError: operands could not be broadcast together with shapes (3,4) (4,1) 

If you did this correctly, the values should be the same as above.

TODO:  

Print out the attention matrix
You will see that the values are quite extreme (one is very close to one and the others are very close to zero.  Now we'll fix this problem by using scaled dot-product attention.

In [43]:
def scaled_dot_product_self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):
    """
    Compute scaled dot-product self-attention mechanism in matrix form.

    Parameters:
    - X : Input matrix (NxD) where N is the number of inputs, D is the dimension of each input.
    - omega_v, omega_q, omega_k : Weight matrices for values, queries, and keys, respectively.
    - beta_v, beta_q, beta_k : Bias vectors for values, queries, and keys, respectively.

    Returns:
    - X_prime : Output matrix after applying scaled dot-product self-attention (NxD).
    """

    # Step 1: Compute queries, keys, and values
    queries = X @ omega_q + beta_q   # Shape: (N, D)
    keys = X @ omega_k + beta_k      # Shape: (N, D)
    values = X @ omega_v + beta_v    # Shape: (N, D)

    # Step 2: Compute dot products between queries and keys (Attention Scores)
    attention_scores = queries @ keys.T   # Shape: (N, N)

    # Step 3: Scale the dot products
    d_k = keys.shape[1]  # Dimension of the key vectors
    scaled_attention_scores = attention_scores / np.sqrt(d_k)

    # Step 4: Apply softmax to calculate attention weights
    attention_weights = softmax(scaled_attention_scores)  # Shape: (N, N)

    # Step 5: Compute weighted sum of values based on attention weights
    X_prime = attention_weights @ values  # Shape: (N, D)

    return X_prime

In [42]:
X_prime = scaled_dot_product_self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

ValueError: operands could not be broadcast together with shapes (3,4) (4,1) 

In self-attention, being "covariant with respect to permutation" means that if we rearrange the columns of the input matrix, the output matrix should rearrange its columns in the same way. This would happen if the output only depended on the relative positions of elements in the input, rather than their fixed positions. However, self-attention usually relies on learned weights, so it is not naturally "permutation covariant" — it’s sensitive to the original order of the sequence. As a result, if we rearrange the input, we will likely get a different output, unless additional mechanisms are added to make the model recognize and handle different orders.






